# テストの実行

In [1]:
import os

import time
import datetime

import subprocess
import logging
from concurrent import futures

import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
TESTSET_DIR = os.path.join('/home', 'jovyan', 'work', '01_testset')
PRJ_DIR = os.path.join('/home', 'jovyan', 'work')

## マスタの読み込み

In [4]:
pre_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '01_testset_pre_master.csv'), usecols=['seed'])
sys_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '02_testset_sys_master.csv'), usecols=['seed'])
stress_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '03_testset_stress_master.csv'), usecols=['seed'])
param_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '04_testset_param_master.csv'), usecols=['seed'])

## 相対スコア用にChampionDataの読み込み

In [5]:
CHAMP_TAG = 'yyyymmdd_hhmm'
CHAMP_DIR = os.path.join('/home', 'jovyan', 'work', 'result', 'champion')

champ_path = os.path.join(CHAMP_DIR, 'champ_all_{}.csv'.format(CHAMP_TAG))
champ_df = pd.read_csv(champ_path)

top_rate = 1.00  # 順位表を参考にチャンピオンスコアを補正
champ_score_dict = {}

for _, row in champ_df.iterrows():
    seed = row['seed']
    score = row['champion_score']

    champ_score_dict[seed] = score

## 実行するロジックの指定

In [7]:
# 実行プログラムにタグをつけておく
PROG_TAG = 'final'

In [8]:
def solve(seed, D, problem, prog_path):
    problem_path = os.path.join(TESTSET_DIR, 'in', '{:0>4}.txt'.format(seed)) 
    command_str = 'cat {} | {}'.format(problem_path, prog_path)

    start_time = time.perf_counter()
    res = subprocess.run(command_str, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

    # 経過時間(ミリ秒単位)
    e_time = time.perf_counter() - start_time
    e_time = int(1000 * e_time)    
    
    #print('{}'.format(prob_id))    
    return (seed, D, e_time, res)

In [9]:
def run_test(testset_name):
    result_df = pd.DataFrame()
    future_list = []

    logger.info('Start')

    testset_path = os.path.join(TESTSET_DIR, testset_name + '_master.csv')
    testset_df = pd.read_csv(testset_path)

    with futures.ThreadPoolExecutor(max_workers=24) as executor:
        for _, row in testset_df.iterrows():
            seed, D, problem = row

            # バッチ実行
            future = executor.submit(solve, seed=seed,  D=D, problem=problem, prog_path=prog_path)
            future_list.append(future)

        _ = futures.as_completed(fs=future_list)

    for future in future_list:
        seed, D, e_time, res = future.result()

        # 結果をまとめる
        solve_result = []
        
        solve_result.append(testset_name)

        # 問題パラメタ
        solve_result.append(seed)
        solve_result.append(D)

        # 経過時間
        solve_result.append(e_time)
        
        try:
            elem_cnt = 2
            
            # Result
            result = res.stderr.decode('utf-8').split()[-elem_cnt].replace('Result=', '')

            # Score
            score = int(res.stderr.decode('utf-8').split()[-elem_cnt+1].replace('Score=', ''))
            
            # 相対スコア
            # rel_score = int(10 ** 9 * top_rate * champ_score_dict[seed] / score)

        except Exception as e:
            print('Error: seed={}'.format(seed))
            print(e)
            return

        solve_result.append(score)
        # solve_result.append(rel_score)

        solve_result.append(result)

        result_df = pd.concat([result_df, pd.DataFrame(solve_result).T], axis=0)

    logger.info('finish!')
    
    # 結果を整形
    result_df.index = range(result_df.shape[0])
    result_df.columns = ['testset', 'seed', 'D', 'time', 'score', 'result']

    # 自己相対スコアを算出
    
    return result_df

In [10]:
logger = logging.getLogger(__name__)

fmt = "%(asctime)s: %(message)s"
logging.basicConfig(level=logging.INFO, format=fmt)

In [11]:
def get_summary_df(result_df):
    # 全体サマリ
    summary_all_df = pd.DataFrame()

    for testset in np.unique(result_df['testset']):
        test_result_df = result_df.query('testset == "{}"'.format(testset))

        summary_df = pd.DataFrame(
        {
            'testset': [testset],
            
            'time_mean': [int(np.mean(test_result_df['time']))],
            
            'score_mean': [int(np.mean(test_result_df['score']))],
            'score_min': [min(test_result_df['score'])],
            'score_max': [max(test_result_df['score'])],

            'time_max': [max(test_result_df['time'])],
        })

        summary_all_df = pd.concat([summary_all_df, summary_df], axis=0)   

    summary_all_df['tag'] = PROG_TAG
    
    cols = ['tag']
    cols.extend(summary_df.columns)
    
    summary_all_df = summary_all_df[cols]
    
    return summary_all_df

In [12]:
PROG_NAME_LIST = ['main']
#PROG_NAME_LIST = ['main', 'main_off']

#testset_name = '01_testset_pre'
#testset_name = '02_testset_sys'
#testset_name = '03_testset_stress'
testset_name = '04_testset_param'

result_dict = {}
summary_all_dict = {}

for PROG_NAME in PROG_NAME_LIST:
    prog_path = os.path.join(PRJ_DIR, PROG_NAME)
    
    result_df = pd.DataFrame()
    
    testset_result_df = run_test(testset_name)
    result_df = pd.concat([result_df, testset_result_df], axis=0)
    
    result_dict[PROG_NAME] = result_df
    summary_all_dict[PROG_NAME] = get_summary_df(result_df)    

2023-04-02 07:44:31,378: Start
2023-04-02 07:44:58,559: finish!


In [13]:
# pre test
result_sub_df = pd.merge(pre_seed_df, result_df, on='seed')
summary_df = get_summary_df(result_sub_df)
summary_df['testset'] = '01_test_pre'

summary_df

,tag,testset,time_mean,score_mean,score_min,score_max,rel_score_mean,rel_score_min,rel_score_max,block_cnt_mean,block_cnt_min,block_cnt_max,disuse_block_size_mean,disuse_block_size_min,disuse_block_size_max,size01_mean,size04_mean,size09_mean,size29_mean,size30_mean,time_max
0,final,01_test_pre,3006,2007716590,58115133,25325931013,458329424,95874624,740866992,8.76,2,38,0.4,0,10,0.94,1.44,2.12,2.86,1.4,3018


In [14]:
# sys param
summary_df = pd.DataFrame()

if testset_name == '04_testset_param' or '02_testset_sys' or testset_name == '03_testset_stress':
    result_sub_df = pd.merge(param_seed_df, result_df, on='seed')
    summary_df = get_summary_df(result_sub_df)
    summary_df['testset'] = '04_test_param'

summary_df

,tag,testset,time_mean,score_mean,score_min,score_max,rel_score_mean,rel_score_min,rel_score_max,block_cnt_mean,block_cnt_min,block_cnt_max,disuse_block_size_mean,disuse_block_size_min,disuse_block_size_max,size01_mean,size04_mean,size09_mean,size29_mean,size30_mean,time_max
0,final,04_test_param,3014,1347506073,58115133,25325931013,437414218,83615142,2725453367,7.915,2,38,0.155,0,10,0.39,1.245,2.14,2.695,1.445,3086


In [25]:
# sys test
summary_df = pd.DataFrame()

if testset_name == '02_testset_sys' or testset_name == '03_testset_stress':
    result_sub_df = pd.merge(sys_seed_df, result_df, on='seed')
    summary_df = get_summary_df(result_sub_df)
    summary_df['testset'] = '02_test_sys'

summary_df

,tag,testset,time_mean,score_mean,score_min,score_max,rel_score_mean,rel_score_min,rel_score_max,block_cnt_mean,block_cnt_min,block_cnt_max,disuse_block_size_mean,disuse_block_size_min,disuse_block_size_max,size01_mean,size04_mean,size09_mean,size29_mean,size30_mean,time_max
0,beam-dbl-state,02_test_sys,3004,1393793564,37917121,50557936508,404974304,16617873,1082190324,7.931,2,61,0.314,0,40,0.453,1.1905,2.161,2.6475,1.479,3103


In [26]:
summary_df[['time_mean', 'score_mean', 'time_max']]

,time_mean,score_mean,time_max
0,3004,1393793564,3103


In [27]:
# stress test
summary_df = pd.DataFrame()

if testset_name == '03_testset_stress':
    result_sub_df = pd.merge(stress_seed_df, result_df, on='seed')
    summary_df = get_summary_df(result_sub_df)
    summary_df['testset'] = '03_test_stress'

summary_df

""


In [28]:
result_df.sort_values('block_cnt', ascending=True)

,testset,seed,D,time,score,rel_score,block_cnt,disuse_block_size,size01,size04,size09,size29,size30,result
1960,02_testset_sys,1791,5,3002,214285714,650000000,2,0,0,0,1,1,0,2R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-2-1-1-0-0-2-1-...
1640,02_testset_sys,1478,5,3002,208333333,650000000,2,0,0,0,1,1,0,2R0-0-0-1-1-0-0-0-0-0-2-2-0-0-0-0-2-2-0-0-0-0-...
180,02_testset_sys,148,5,3002,187500000,577777778,2,0,0,0,1,1,0,2R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...
620,02_testset_sys,516,5,3003,182539683,650000000,2,0,0,0,1,1,0,2R0-0-0-0-0-0-0-0-0-0-0-2-2-0-0-2-2-2-1-0-2-2-...
820,02_testset_sys,743,5,3002,171428571,597474750,2,0,0,0,0,2,0,2R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,02_testset_sys,845,14,3005,9321256039,393635257,38,0,2,11,21,4,0,38R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0...
47,02_testset_sys,63,12,3006,25325931013,488190893,38,10,22,6,4,6,0,38R0-0-0-0-29-30-0-0-0-0-0-1-0-0-0-0-0-0-0-0-0...
269,02_testset_sys,376,14,3005,10757661271,418432982,38,0,2,18,8,10,0,38R0-0-0-25-0-0-0-0-0-0-0-0-0-0-0-0-19-0-0-0-0...
78,02_testset_sys,117,13,3004,36141245636,404287624,43,34,35,3,1,2,2,43R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0...


In [29]:
result_df.sort_values('time', ascending=True)

,testset,seed,D,time,score,rel_score,block_cnt,disuse_block_size,size01,size04,size09,size29,size30,result
80,02_testset_sys,58,5,3001,154761905,650000000,2,0,0,0,0,2,0,2R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...
1852,02_testset_sys,1878,7,3001,845565954,457817116,6,0,0,1,1,4,0,6R0-0-0-0-3-3-3-0-0-0-6-6-3-0-0-0-0-6-0-3-0-0-...
1102,02_testset_sys,1135,7,3001,552020202,535407136,4,0,0,1,1,2,0,4R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...
101,02_testset_sys,144,6,3001,266666667,627840909,3,0,0,0,0,3,0,3R0-0-0-3-0-0-0-0-3-1-1-1-2-2-1-1-1-1-0-2-0-0-...
190,02_testset_sys,149,5,3001,182539683,610434779,2,0,0,0,1,1,0,2R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,02_testset_sys,1364,5,3050,434523810,614383561,3,0,0,0,3,0,0,3R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...
1489,02_testset_sys,1583,14,3053,516166646,303377275,8,0,0,0,1,4,3,8R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...
22,02_testset_sys,22,7,3055,1635353535,470257765,7,0,0,4,1,2,0,7R0-0-0-0-0-0-0-0-0-0-0-0-2-0-0-0-0-0-0-0-0-0-...
1488,02_testset_sys,1741,13,3056,235420512,308836391,7,0,0,0,0,3,4,7R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...


In [30]:
result_df.sort_values('rel_score', ascending=True).head(n=10)

,testset,seed,D,time,score,rel_score,block_cnt,disuse_block_size,size01,size04,size09,size29,size30,result
1079,02_testset_sys,1069,14,3006,8141095809,16617873,15,7,7,2,1,1,4,15R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0...
539,02_testset_sys,612,14,3005,873453141,35223418,10,0,0,1,3,3,3,10R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0...
909,02_testset_sys,895,14,3006,340710296,47229200,7,0,0,0,0,4,3,7R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...
286,02_testset_sys,322,11,3003,1215143871,48297469,8,0,0,2,2,1,3,8R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...
1504,02_testset_sys,1321,9,3032,516355641,52181949,6,0,0,0,3,1,2,6R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...
1679,02_testset_sys,1799,14,3007,5268924666,53136330,14,3,4,3,0,4,3,14R0-1-1-1-1-1-1-1-1-1-1-0-0-0-0-1-1-1-1-1-1-1...
939,02_testset_sys,964,14,3005,101059467,55601850,3,0,0,0,0,1,2,3R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...
356,02_testset_sys,400,11,3003,3612281885,56399817,10,1,2,3,1,2,2,10R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0...
665,02_testset_sys,562,10,3003,1733087633,61239542,9,0,0,3,2,2,2,9R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...
1986,02_testset_sys,2044,11,3004,2940912421,66545485,9,2,2,1,2,2,2,9R0-0-1-1-1-1-1-1-1-0-0-1-1-1-1-1-1-1-1-1-0-0-...


# 結果ログの保存

In [33]:
t_now = datetime.datetime.now() + datetime.timedelta(hours=9)
time_str = t_now.strftime('%Y%m%d_%H%M')

for PROG_NAME in PROG_NAME_LIST:
    result_df = result_dict[PROG_NAME]
    
    for testset in np.unique(result_df['testset']):
        csv_df = result_df.query('testset == "{}"'.format(testset))
        csv_df.to_csv(PRJ_DIR+'/result/{}_{}_{}_{}.csv'.format(time_str,PROG_TAG, testset, PROG_NAME), index=False)